In [0]:
import tensorflow as tf
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense
from keras.models import model_from_json
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt

In [2]:
uploaded  = files.upload()




Saving data.csv to data.csv


In [75]:
for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "data.csv" with length 301072766 bytes


In [0]:
config= tf.ConfigProto( device_count = {'GPU': 0 , 'CPU':40})
session= tf.Session(config=config) 

keras.backend.set_session(session)

In [0]:
with open("data.csv") as f:
    dataset = f.readlines()
    dataset_array=np.array(dataset)
    
   


In [0]:
number_of_classes=7 
batch_size = 256
epochs = 5


In [0]:
# Dividing data into training and test set category as per dataset's third column
x_train =[]
y_train =[]
x_test =[]
y_test = []


for i in range(1,dataset_array.size):
  
        emotion, pixels_list, usage = dataset_array[i].split(",")
          
        val = pixels_list.split(" ")
            
        pixels = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, number_of_classes)
            
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
  


In [80]:

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train=preprocessing.normalize(x_train)
x_test=preprocessing.normalize(x_test)
print('training samples shape',x_train.shape)
print('testing samples shape', x_test.shape)

('training samples shape', (28709, 2304))
('testing samples shape', (3589, 2304))


In [0]:
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')



In [0]:
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

In [0]:
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(number_of_classes, activation='softmax'))

In [0]:
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

#------------------------------

model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)


In [85]:
fit = True

if fit == True:
	
	model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs) 
else:
	model.load_weights('mod.h5')

Epoch 1/5
256/256 [==============================] - 541s 2s/step - loss: 1.8162 - acc: 0.2493
Epoch 2/5
256/256 [==============================] - 536s 2s/step - loss: 1.8113 - acc: 0.2522
Epoch 3/5
102/256 [==========>...................] - ETA: 5:19 - loss: 1.8122 - acc: 0.2480

256/256 [==============================] - 528s 2s/step - loss: 1.8114 - acc: 0.2504
Epoch 4/5
256/256 [==============================] - 531s 2s/step - loss: 1.8120 - acc: 0.2508
Epoch 5/5
155/256 [=================>............] - ETA: 3:26 - loss: 1.8128 - acc: 0.2497

256/256 [==============================] - 526s 2s/step - loss: 1.8108 - acc: 0.2508


In [87]:
#overall evaluation
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', 100*score[1])


3589/3589 [==============================] - 9s 2ms/step
('Test loss:', 1.8110974573697112)
('Test accuracy:', 24.937308442878273)


In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")




In [46]:
!ls

data.csv  datalab  model.h5  model_json  model.json


In [0]:


files.download("model.json")

In [0]:
files.download("model.h5")